## Data processing for dashboard

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
from datetime import datetime

#import matplotlib.pyplot as plt
#import seaborn as sns

#from sklearn.preprocessing import MinMaxScaler
#import geopandas as gpd

## Read in and process dependent variable - turnover

Read turnover data from csv into a dataframe.

In [ ]:
# annual and monthly data

annual_url = '../01_data/processed_annual_turnover.csv'
#monthly_url = '../01_data/processed_monthly_turnover.csv'

annual_df = pd.read_csv(annual_url, parse_dates=['month_year'])
annual_df = annual_df.drop(['n'],axis=1)
#annual_df.info()

#monthly_df = pd.read_csv(monthly_url, parse_dates=['month_year'])
#monthly_df = monthly_df.drop(['n'],axis=1)
annual_df.info()


Function to clean and process DV dataframes

In [ ]:
def clean_dv(df):
    # drop org_type because it has historic variation which is creating duplicates when mapping later
    df.drop('org_type', axis=1, inplace=True) 
    # add leaver and joiner rates column
    df['leaver_rate'] = df['leave_FTE']/df['denom_FTE_average']
    df['joiner_rate'] = df['join_FTE']/df['denom_FTE_average']

    # replace inf values with nan (can happen with rate calcs)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    return df

In [ ]:
annual_df1 = clean_dv(annual_df)
#monthly_df1 = clean_dv(monthly_df)

In [ ]:
annual_df1 = annual_df1.sort_values(by='month_year')

In [ ]:
# cut first year of dataframe as this only contains start denoms values
annual_df2 = annual_df1[annual_df1['month_year'] >= (annual_df1['month_year'].min()) + pd.DateOffset(years=1)]

In [ ]:
annual_df2.head()

The data show the full time equivalent (FTE) and headcount (HC) number of leavers by organisation and staff group for the previous 12-month period from the date.

In [ ]:
annual_df2['staff_group'].unique()

## Load independent variable 1 - local unemployment

Load data about local unemployment.

In [ ]:
url_r1 = '../01_data/ONS_localunemployment_monthly.csv'
df_r1 = pd.read_csv(url_r1, parse_dates=['Date'])

#df_r1.drop(['thousands'],axis=1,inplace=True)
df_r1.rename(columns={'%':'local_unemployment','Date':'month_year',
                      'NHSE region name':'region_name'},inplace=True)
df_r1 = df_r1.sort_values('month_year')
df_r1.tail()

## Load IV 2 - sickness absence

Load data about sickness absence to use as second regressor

In [ ]:
url_r2 = '../01_data/data_wrangling/sickness_benchmarking.csv'
df_r2 = pd.read_csv(url_r2, parse_dates=['DATE'])

In [ ]:
df_r2['BENCHMARK_GROUP'].unique()

In [ ]:
trust_types_todrop = ['Clinical Commissioning Group','Integrated Care Board']
df_r2 = df_r2[~df_r2['CLUSTER_GROUP'].isin(trust_types_todrop)]
df_r2 = df_r2.drop(['BENCHMARK_GROUP','ORG_NAME',
                    'NHSE_REGION_CODE','CLUSTER_GROUP','file_date'],axis=1)
df_r2.rename(columns={'ORG_CODE':'org_code','DATE':'month_year',
                      'NHSE_REGION_NAME':'region_name','STAFF_GROUP':'staff_group',
                      'FTE_DAYS_LOST':'fte_days_lost','FTE_DAYS_AVAILABLE':'fte_days_available'},inplace=True)
merge_cols = ['month_year', 'org_code','region_name','staff_group']
df_r2['sickness_absence'] = df_r2['fte_days_lost']/df_r2['fte_days_available']
df_r2 = df_r2.reset_index(drop=True)
df_r2.info()

In [ ]:
df_r2.tail()

Create 12-month rolling sickness absence column for use with annual turnover data

In [ ]:
df_r2['month_year'] = pd.to_datetime(df_r2['month_year'])

# Sort the DataFrame by organisation, staff_group, and month
df_r2.sort_values(by=['org_code', 'staff_group', 'month_year'], inplace=True)

# Calculate the rolling sums for days lost and days available
df_r2['rolling_days_lost'] = df_r2.groupby(['org_code', 
                        'staff_group'])['fte_days_lost'].rolling(window=12, min_periods=1).sum().reset_index(level=[0, 1], drop=True)

df_r2['rolling_days_available'] = df_r2.groupby(['org_code', 
                        'staff_group'])['fte_days_available'].rolling(window=12, min_periods=1).sum().reset_index(level=[0, 1], drop=True)

# Calculate the rolling sickness absence rate
df_r2['annual_sickness_absence'] = df_r2['rolling_days_lost'] / df_r2['rolling_days_available']

# drop fte_days_lost fte_days_available, rolling_days_available and rolling_days_lost columns
#df_r2.drop(columns=['fte_days_lost', 'fte_days_available', 'rolling_days_available', 'rolling_days_lost'], inplace=True)

df_r2.head()

In [ ]:
a = sorted(df_r2['staff_group'].unique())
a


In [ ]:
b = []
len(b)

In [ ]:
# Get the top 4 staff groups based on average absence rate
top_staff_groups = df_r2.groupby('staff_group')['sickness_absence'].mean().nlargest(4).index

# Filter the DataFrame for the top 4 staff groups
top_groups_df = df_r2[df_r2['staff_group'].isin(top_staff_groups)]

# Pivot the DataFrame to have staff groups as columns
pivoted_df = top_groups_df.pivot_table(index='month_year', columns='staff_group', values='sickness_absence', aggfunc='mean')

In [ ]:
pivoted_df

In [ ]:
df_r2.groupby(['month_year', 'staff_group'])['sickness_absence'].mean().reset_index()

In [ ]:
import matplotlib.pyplot as plt

# Plotting each staff group as a separate line
plt.figure(figsize=(10, 6))

for staff_group in pivoted_df.columns:
    plt.plot(pivoted_df.index, pivoted_df[staff_group], marker='o', linestyle='-', label=staff_group)

plt.title('Sickness Absence Rate Over Time for All Staff Groups')
plt.xlabel('Date')
plt.ylabel('Absence Rate (%)')
plt.legend()
plt.show()

## Load IV 3 - reasons for sickness absence

Add data about reasons for sickness absence

In [ ]:
url_r3 = '../01_data/sickness_absence_reason_pivot.csv'
df_r3 = pd.read_csv(url_r3, parse_dates=['Date'])
#df_r3 = df_r3.drop(['FTE days lost'],axis=1)
df_r3.rename(columns={'Date':'month_year','Staff group':'staff_group'},inplace=True)
#df_r2 = df_r2.reset_index(drop=True)

# drop least frequent reasons for absence
df_r3 = df_r3.drop(['substance_abus','asthma',
                    'dental','blood_disorder','endocrine',
                    'eye','skin_disorders','nervous_system'],axis=1)

# Replace NaN values with 0 
df_r3 = df_r3.fillna(0)
df_r3.info()
# national level data

In [ ]:
df_r3.tail()

## Load IV 4 and 5 - staff vacancies

In [ ]:
url_sg_ref = '../01_data/ref_sg_vacancy.csv'
df_sg_ref = pd.read_csv(url_sg_ref)
df_sg_ref.head()

In [ ]:
url_r4 = '../01_data/vacancy_ESR.csv'
df_r4 = pd.read_csv(url_r4,parse_dates=['month_year'],dayfirst=True)

df_r4 = df_r4.drop(['Published month','Published quarter','England'],axis=1)
df_r4.rename(columns={'NWD Staff Group':'vacancy_sg','NHS England region':'region_name',
                        'Vacancy Wte':'vacancy_FTE'},inplace=True)

df_r4 = df_r4.fillna(0)

# Remove code in brackets
df_r4['region_name'] = df_r4['region_name'].str[:-6].str.rstrip()

# Add staff groupings to match other datasets
df_r4 = pd.merge(df_r4, df_sg_ref, on='vacancy_sg',how='left')

df_r4 = df_r4.drop(['all'],axis=1)

df_r4.info()

# regional level

In [ ]:
url_r5 = '../01_data/vacancy_TRAC.csv'
df_r5 = pd.read_csv(url_r5,parse_dates=['month_year'],dayfirst=True)

df_r5 = df_r5.drop(['Published month','Published quarter','England'],axis=1)

df_r5.rename(columns={'NWD Staff Group':'vacancy_sg','NHS England region':'region_name',
                        'Advertised FTE':'advertised_FTE'},inplace=True)

df_r5 = df_r5.fillna(0)

# Remove region code in brackets
df_r5['region_name'] = df_r5['region_name'].str[:-6].str.rstrip()

# Add staff groupings to match other datasets
df_r5 = pd.merge(df_r5, df_sg_ref, on='vacancy_sg',how='left')

df_r5 = df_r5.drop(['all'],axis=1)

# regional level

df_r5.info()

## Load IV 6 - Reasons for leaving

Data are only quarterly

In [ ]:
url_r6 = '../01_data/rfl_dec22.csv'
df_r6 = pd.read_csv(url_r6,parse_dates=['month_year'],dayfirst=True)

#df_r6 = df_r6.drop(['financial_year','quarter'],axis=1)

# df_r5.rename(columns={'NWD Staff Group':'staff_group','NHS England region':'region_name',
#                         'Advertised FTE':'advertised_FTE'},inplace=True)

# df_r5 = df_r5.fillna(0)

# # Remove region code in brackets
# df_r5['region_name'] = df_r5['region_name'].str[:-6].str.rstrip()
# # regional level

# shortern column names and add %_ at beginning
df_r6.rename(columns={'Death in service':'%_death_in_service','Dismissal':'%_dismissal',
                        'End of fixed term':'%_end_of_ft','Flexibility':'%_flexibility',
                        'Health':'%_health','Incompatible working relationships':'%_incompat_relations',
                        'Other':'%_other', 'Pay/Reward':'%_pay_reward', 'Pregnancy':'%_pregnancy',
                        'Progression/CPD':'%_progression_cpd','Relocation':'%_relocation',
                        'Retirement':'%_retirement','Unknown':'%_unknown','Work/Life Balance':'%_work_life_balance',
                        'Workforce Transformation':'%_workforce_transform'},inplace=True)

df_r6['%_other'] = pd.to_numeric(df_r6['%_other'], errors='coerce')
df_r6['%_unknown'] = pd.to_numeric(df_r6['%_unknown'], errors='coerce')
df_r6['%_workforce_transform'] = pd.to_numeric(df_r6['%_workforce_transform'], errors='coerce')

df_r6.info()

## Merge IV dfs to main df

In [ ]:
annual_df1.head()

In [ ]:
def merge_ivs(df, df_r1, df_r2, df_r3,df_r4,df_r5,df_r6):
    # local unemployment rate
    df1 = pd.merge(df, df_r1, on=['month_year', 'region_name'],how='left')
    df1 = df1.sort_values('month_year')

    # sickness absence
    r2_merge_cols = ['month_year', 'org_code','region_name','staff_group']
    df2 = pd.merge(df1, df_r2, on=r2_merge_cols,how='left')
    #df2.drop_duplicates(subset=r2_merge_cols)

    # reason for sickness absence
    r3_merge_cols = ['month_year','staff_group']
    df3 = pd.merge(df2, df_r3, on=r3_merge_cols,how='left')

    # vacancy
    r4_merge_cols = ['month_year','region_name','staff_group']

    df4 = pd.merge(df3, df_r4, on=r4_merge_cols,how='left')

    # df4.drop(columns=['vacancy_sg'], inplace=True)

    # df4['vacancy_rate'] = df4['vacancy_FTE'] / df4['sip_FTE_region']

    # advertised vacancy
    df5 = pd.merge(df4, df_r5, on=r4_merge_cols,how='left')

    # df5.drop(columns=['vacancy_sg'], inplace=True)

    # df5['advertised_rate'] = df5['advertised_FTE'] / df5['sip_FTE_region']

    # df5.drop(columns=['sip_FTE_region','advertised_FTE','vacancy_FTE'], inplace=True)

    # reasons for leaving
    df6 = pd.merge(df5, df_r6, on='month_year',how='left')

    # Need to cut dataframe to earliest and latest data available for all fields. Do this by cutting rows where all values for key variables are zero
    df6 = df6[~((df5['leaver_rate'] == 0) | (df6['joiner_rate'] == 0) | (df6['sickness_absence'] == 0))]
    max_date = df6['month_year'].max()
    min_date = df6['month_year'].min()
    # Calculate the difference in months
    delta = relativedelta(max_date, min_date)
    # Extract the number of months
    months_difference = delta.years * 12 + delta.months
    print(f"After trimming, the dataframe ranges from {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}, "
           f"giving {months_difference} months of data")
    # transform nans into 0s
    df6 = df6.fillna(0)

    # drop duplicates
    df6.drop_duplicates(inplace=True)
    
    return df6
#    return max_date

In [ ]:
annual_df_ivs = merge_ivs(annual_df1,df_r1, df_r2, df_r3,df_r4,df_r5,df_r6)

In [ ]:
# check sickness_absence

In [ ]:
#monthly_df_ivs = merge_ivs(monthly_df1,df_r1, df_r2, df_r3,df_r4,df_r5,df_r6)

In [ ]:
annual_df_ivs.tail()

In [ ]:
annual_df_ivs.info()

In [ ]:
annual_df_ivs.to_csv(f'annual_dash_data.csv', index=False)

In [ ]:
#monthly_df_ivs.to_csv(f'monthly_dash_data.csv', index=False)